In [1]:
import os
from dotenv import load_dotenv

import hashlib
import pandas as pd
import pandahouse as ph

In [2]:
load_dotenv()
# Подключение к базе
connection = {
    'host': os.getenv('HOST'),
    'password': os.getenv('PASSWORD'),
    'user': os.getenv('DB_USER')
}

## Разделение на группы в Pandas

Разбиваем пользователей на 5 групп

In [3]:
def ab_split(id, salt='exp_mess_1', n_groups=5):
    """Функция разбивает выборку на группы 
    с применением хэширования с солью

    Args:
        id (int | str): id пользователя
        salt (int | str, optional): Соль. Defaults to 'exp_mess_1'.
        n_groups (int, optional): Количество групп для разбиения. Defaults to 5.

    Returns:
        int: Номер группы
    """
    # Объединяем id c солью
    test_id = str(id) + '-' + str(salt) 
    # Вычисляем hash от этой строки
    test_id_deigest = hashlib.md5(test_id.encode('ascii')).hexdigest()
    # Преобразуем hash в целое число
    test_id_final_int = int(test_id_deigest, 16)
    # Возвращаем номер группы - остаток от деления на количество групп
    return test_id_final_int % n_groups

Выгрузим пользователей, разделим на группы и сравним получившееся количество пользователей в группах между собой.

In [4]:
q = """
SELECT 
    DISTINCT user_id
FROM simulator_20240420.feed_actions
WHERE toDate(time) BETWEEN '2024-03-29' AND '2024-04-04'
"""

users_df = ph.read_clickhouse(q, connection=connection)
users_df

,user_id
0,123767
1,129655
2,131026
3,28786
4,135485
...,...
49809,121332
49810,25551
49811,29193
49812,107073


In [6]:
# Готовим группы
users_df['hash_group'] = users_df.user_id.apply(ab_split)
# Число пользователей в каждой группе
users_df.groupby('hash_group').user_id.nunique()

hash_group
0    10073
1     9877
2     9893
3     9939
4    10032
Name: user_id, dtype: int64

Получилось пять групп, количество пользователей приблизительно одинаковое.

## Разделение на группы в ClickHouse

In [8]:
q_2 = """
SELECT xxHash64(toString(user_id)||'my_salt')%5 as group, 
    uniqExact(user_id)
FROM simulator_20240420.feed_actions 
WHERE toDate(time) BETWEEN '2024-03-29' AND '2024-04-04' 
group by group
"""

In [9]:
ph.read_clickhouse(q_2, connection=connection)

,group,uniqExact(user_id)
0,0,10027
1,1,9866
2,2,9916
3,3,10084
4,4,9921


Также видим пять групп с приблизительно одинаковым количеством пользователей